【問題1】スクラッチを振り返る

ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったか

・重みの初期化を行うクラス

・活性化関数のクラス

・最適化のクラス

・ロスを計算するクラス

・forwardで内積、backwardで勾配を計算する(全結合層)クラス

・Convolutionを行うクラス

・Poolingを行うクラス

・dropoutを行うクラス

・fitでループを回し更新を行い、predict, accuracyを算出するクラス

を実装した

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="df_vergicolor_and_virginica.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["target"] == "versicolor")|(df["target"] == "virginica")]
y = df["target"]
X = df.loc[:, ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='versicolor'] = 0
y[y=='virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    

Epoch 0, loss : 23.8744, val_loss : 48.7048, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 12.0906, val_loss : 3.1808, acc : 0.500, val_acc : 0.750
Epoch 2, loss : 3.6726, val_loss : 1.2019, acc : 0.750, val_acc : 0.750
Epoch 3, loss : 1.2309, val_loss : 6.0988, acc : 0.750, val_acc : 0.625
Epoch 4, loss : 0.3250, val_loss : 1.5712, acc : 0.750, val_acc : 0.812
Epoch 5, loss : 0.0035, val_loss : 6.6560, acc : 1.000, val_acc : 0.688
Epoch 6, loss : 0.1553, val_loss : 2.6371, acc : 1.000, val_acc : 0.812
Epoch 7, loss : 0.0004, val_loss : 6.9502, acc : 1.000, val_acc : 0.688
Epoch 8, loss : 0.0000, val_loss : 2.1335, acc : 1.000, val_acc : 0.812
Epoch 9, loss : 0.0199, val_loss : 8.7734, acc : 1.000, val_acc : 0.625
test_acc : 0.800


【問題2】スクラッチとTensorFlowの対応を考える

#### 重みの初期化を行うクラス

今回は関数で実装されている

weights = {
        'w': tf.Variable(tf.random_normal([n_input, n_hidden1]))
    }
    
biases = {
       'b': tf.Variable(tf.random_normal([n_hidden1]))
    }
    
#### 活性化関数のクラス
今回は関数

layer = tf.nn.relu(layer)の部分

#### 最適化のクラス
tf.train.の後に指定したい最適化手法を選択します。

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

#### ロスを計算するクラス
分類の目的関数はlogit関数なのを思い出せる作りになっている

softmaxやsigmoidとcross_entropyはセットで考えてるっぽい

loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
#### forwardで内積、backwardで勾配を計算する(全結合層)クラス
layer = tf.add(tf.matmul(x, weights['w1']), biases['b1'])

layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']

個人的には下のがしっくりくるけどカッコいいのは上っぽい

そしてbackwardが自動なのが最高!

#### Convolutionを行うクラス
畳み込みではない
#### Poolingを行うクラス
これは畳み込みではない
#### dropoutを行うクラス
今回はなし
#### fitでループを回し更新を行い、predict, accuracyを算出するクラス
ループ
epochごと
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
    
   ミニバッチごと
       for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
予測       
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
指標値
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [3]:
dataset_path = "Iris.csv"
df = pd.read_csv(dataset_path)
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]
X = np.array(X)
y = np.array(y)
y[y=="Iris-setosa"] = 0
y[y=="Iris-versicolor"] = 1
y[y=="Iris-virginica"] = 2
y = y.astype(np.int)# ndarray(n_samples, n_features)
# onehotencoder
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size = 0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [4]:
#ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 20

n_hidden1 = 50
n_hidden2 = 100
n_samples = X_train.shape[0]
n_input = X_train.shape[1]
n_classes = y_train.shape[1]


X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x, n_input, n_hidden1, n_hidden2, n_classes):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X, n_input, n_hidden1, n_hidden2, n_classes)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.arg_max(Y, 1),tf.arg_max(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# varable の初期化(忘れやすい)
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x,mini_batch_y) in enumerate(get_mini_batch_train):
            #ミニバッチごとにループ
            sess.run(train_op, feed_dict={X:mini_batch_x, Y:mini_batch_y})
            loss,acc = sess.run([loss_op, accuracy], feed_dict={X:mini_batch_x, Y:mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy],feed_dict={X:X_val, Y:y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch,loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X:X_test, Y:y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use `argmax` instead
Epoch 0, loss : 4.7242, val_loss : 2.4979, acc : 0.833, val_acc : 0.833
Epoch 1, loss : 1.9071, val_loss : 12.6026, acc : 0.667, val_acc : 0.667
Epoch 2, loss : 1.5139, val_loss : 14.1973, acc : 0.833, val_acc : 0.625
Epoch 3, loss : 0.0000, val_loss : 6.6303, acc : 1.000, val_acc : 0.708
Epoch 4, loss : 0.0000, val_loss : 2.0461, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0000, val_loss : 1.2028, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 3.2036, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 1.1746, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0000, val_loss : 1.0506, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0000, val_loss : 1.0125, acc : 1.000, val_acc : 0.958
Epoch 10, loss : 0.0000, val_loss : 0.9583, acc : 1.000, val_acc : 0.958
Epoch 11, loss : 0.0000, val_loss : 0.9839, acc : 1.000, val_acc : 0.917
Epoch 12, loss : 0.0000, val_loss : 2.0489, acc : 1.000, val_acc : 0.833
Epoch 13, l

【問題4】House Pricesのモデルを作成

In [ ]:
dataset_path = "df_grlivearea_yearbuilt.csv"
df = pd.read_csv(dataset_path)
X = df.loc[:,['GrLivArea', 'YearBuilt']]
y = df.loc[:,'SalePrice']
X = np.array(X)
y = np.array(y)[:,np.newaxis]
scaler = StandardScaler()
scaler.fit_transform(X)
y_log = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size = 0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
#ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 40

n_hidden1 = 100
n_hidden2 = 50
n_samples = X_train.shape[0]
n_input = X_train.shape[1]
n_output = 1


X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x, n_input, n_hidden1, n_hidden2, n_output):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_output]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_output]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # 回帰はidentity_function
    return layer_output

# ネットワーク構造の読み込み                               
costs = example_net(X, n_input, n_hidden1, n_hidden2, n_output)

# 目的関数
loss_op = tf.sqrt(tf.losses.mean_squared_error(Y, costs))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
predict = costs
# 指標値計算
rmse = loss_op

# varable の初期化(忘れやすい)
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x,mini_batch_y) in enumerate(get_mini_batch_train):
            #ミニバッチごとにループ
            sess.run(train_op, feed_dict={X:mini_batch_x, Y:mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X:mini_batch_x, Y:mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X:X_val, Y:y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch,loss, val_loss))
    test_rmse = sess.run(rmse, feed_dict={X:X_test, Y:y_test})
    print("test_rmse : {:.3f}".format(test_rmse))

Epoch 0, loss : 938.5207, val_loss : 1051.9639
Epoch 1, loss : 4252.5684, val_loss : 4476.3589
Epoch 2, loss : 957.3088, val_loss : 1044.9175
Epoch 3, loss : 796.4057, val_loss : 763.8050
Epoch 4, loss : 2422.5234, val_loss : 2543.3960
Epoch 5, loss : 2700.2554, val_loss : 2915.8201
Epoch 6, loss : 2224.8813, val_loss : 2433.1758
Epoch 7, loss : 868.1229, val_loss : 1174.6713
Epoch 8, loss : 670.2551, val_loss : 604.6128
Epoch 9, loss : 1260.4460, val_loss : 1313.4128
Epoch 10, loss : 321.9333, val_loss : 379.8586
Epoch 11, loss : 239.1192, val_loss : 244.6057
Epoch 12, loss : 660.6674, val_loss : 686.1393
Epoch 13, loss : 185.2248, val_loss : 169.8229
Epoch 14, loss : 676.5880, val_loss : 735.4643
Epoch 15, loss : 1748.9120, val_loss : 1915.2477
Epoch 16, loss : 782.8522, val_loss : 803.9597
Epoch 17, loss : 1033.8152, val_loss : 955.8148
Epoch 18, loss : 310.1647, val_loss : 346.5423
Epoch 19, loss : 482.0647, val_loss : 448.5206
Epoch 20, loss : 635.4265, val_loss : 688.4382
Epoch 2

【問題5】MNISTのモデルを作成

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [ ]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64
print(y_test_one_hot.shape)

(60000,)
(60000, 10)
float64
(10000, 10)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)

(48000, 784)
(12000, 784)


In [ ]:
#ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 100
num_epochs = 20

n_hidden1 = 50
n_hidden2 = 100
n_samples = X_train.shape[0]
n_input = X_train.shape[1]
n_classes = y_train.shape[1]


X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x, n_input, n_hidden1, n_hidden2, n_classes):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X, n_input, n_hidden1, n_hidden2, n_classes)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.arg_max(Y, 1),tf.arg_max(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# varable の初期化(忘れやすい)
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x,mini_batch_y) in enumerate(get_mini_batch_train):
            #ミニバッチごとにループ
            sess.run(train_op, feed_dict={X:mini_batch_x, Y:mini_batch_y})
            loss,acc = sess.run([loss_op, accuracy], feed_dict={X:mini_batch_x, Y:mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy],feed_dict={X:X_val, Y:y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch,loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X:X_test, Y:y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 1.4631, val_loss : 1.7645, acc : 0.560, val_acc : 0.557
Epoch 1, loss : 1.1447, val_loss : 1.1609, acc : 0.680, val_acc : 0.699
Epoch 2, loss : 1.2766, val_loss : 1.0412, acc : 0.720, val_acc : 0.755
Epoch 3, loss : 1.0631, val_loss : 0.8567, acc : 0.790, val_acc : 0.812
Epoch 4, loss : 0.8672, val_loss : 0.7532, acc : 0.770, val_acc : 0.832
Epoch 5, loss : 0.4801, val_loss : 0.6687, acc : 0.840, val_acc : 0.852
Epoch 6, loss : 0.4381, val_loss : 0.6444, acc : 0.860, val_acc : 0.870
Epoch 7, loss : 0.4612, val_loss : 0.6773, acc : 0.890, val_acc : 0.880
Epoch 8, loss : 0.3946, val_loss : 0.6339, acc : 0.880, val_acc : 0.883
Epoch 9, loss : 0.6297, val_loss : 0.5746, acc : 0.880, val_acc : 0.892
Epoch 10, loss : 0.5558, val_loss : 0.5921, acc : 0.870, val_acc : 0.894
Epoch 11, loss : 0.4897, val_loss : 0.5609, acc : 0.880, val_acc : 0.902
Epoch 12, loss : 0.3030, val_loss : 0.5432, acc : 0.910, val_acc : 0.907
Epoch 13, loss : 0.3740, val_loss : 0.5359, acc : 0.900, val_

In [ ]:
tf.summary.FileWriter('val_loss', sess.graph)
!tensorboard --logdir=val_loss --port=8010

<img src="tensorboad.png">